<a href="https://colab.research.google.com/github/uldom/Big_Data_Challenge/blob/main/Level_1/BD_Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.4 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 h

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-24 16:35:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-09-24 16:35:26 (5.96 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
camera_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep='\t', header=True, inferSchema=True)

# Show DataFrame
camera_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [ ]:
camera_df.count()

1801974

In [ ]:
# Drop null values
camera_df = camera_df.dropna()
camera_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [ ]:
camera_df.count()

1801849

In [ ]:
count = camera_df.groupBy('customer_id').count().orderBy('customer_id')
count.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|      10022|    2|
|      10113|    1|
|      10141|    1|
|      10227|    1|
|      10280|    1|
|      10293|    1|
|      10313|    1|
|      10322|    2|
|      10328|   10|
|      10468|    1|
|      10469|    5|
|      10503|    1|
|      10696|    2|
|      10717|    2|
|      10719|    1|
|      10835|    1|
|      10836|    3|
|      11049|    1|
|      11067|    1|
|      11125|    2|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Create customers df for SQL table
customers = count.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10022|             2|
|      10113|             1|
|      10141|             1|
|      10227|             1|
|      10280|             1|
|      10293|             1|
|      10313|             1|
|      10322|             2|
|      10328|            10|
|      10468|             1|
|      10469|             5|
|      10503|             1|
|      10696|             2|
|      10717|             2|
|      10719|             1|
|      10835|             1|
|      10836|             3|
|      11049|             1|
|      11067|             1|
|      11125|             2|
+-----------+--------------+
only showing top 20 rows



In [ ]:
customers.count()

1116681

In [ ]:
#Create products df for SQL table

products = camera_df.select(['product_id', 'product_title']).distinct()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L9ZF5BY|Flylinktech Actio...|
|B00017LSF8|B+W Clear UV Haze...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B002POMZWU|Usb cable for kod...|
|B00FEY2BCW|Camera Case Eveca...|
|B0131XNIDS|*5-pack* microSue...|
|B004OUMTT2|Celestron PowerSe...|
|B004HY3XOA|58MM UV Lens Filt...|
|B00LBFKLRU|(Create a generic...|
|B007QXZ6D0|Vello Cinch Strap...|
|B00PG36JTW|Tamrac TEK Fitted...|
|B003JNJDGM|TTL Off Camera Sh...|
|B00ZYATKCA|Olympus Air Mirro...|
|B00S2X48IW|Qkoo Selfie Handh...|
|B00VQJASPG|    Canon SL1 Bundle|
|B00554PCDG|Pixel Flash Hot S...|
|B00006JPWW|Panasonic AY-DVMC...|
|B00B9Z23NM|Nikon 8250 ACULON...|
|B00GOJ7JGY|Fotasy HET63 Lens...|
|B00IUU22WG|White Photography...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products.count()

168668

In [ ]:
#Create review_id df for SQL table

review = camera_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
review.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407| 2015-08-31|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334| 2015-08-31|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742| 2015-08-31|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315| 2015-08-31|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657| 2015-08-31|
|R2TZNSA18V7YF6|    3183883|B005C95NM4|     246957815| 2015-08-31|
|R22ZVRDPPXIDNL|   23208852|B00LBIBYBE|     746593019| 2015-08

In [ ]:
review.count()

1801849

In [ ]:
# Create vine df for SQL table

vine = camera_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
|R3NPIFKLR19NQA|          3|            0|          0|   N|
|R3MBE6UCH3435E|          3|            8|          8|   N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|
|R1YND4BS823GN5|          1|            0|          0|   N|
|R2TZNSA18V7YF6|          4|            1|          1|   N|
|R22ZVRDPPXIDNL|          5|            0|          0|   N|
|R1F4O6WOO2W461|          3|            3|          3|   N|
| RT1KLS3QSJNUT|          5|            

In [ ]:
vine.count()

1801849

In [ ]:
# Configure settings for RDS

mode = "append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/Big_Data_Camera"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS

review.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS

vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)